# **AI Storytelling**

<img src="assets/logo.jpg" width="500" height="300">

This is Natural Language Processing platform that targets on conversion of short stories to audiobooks with features:
- characters extraction from text,
- voice generation by dialogues, narration and characters,
- musical background creation by text,
- activities sound generation by lines, and
- combination of all above features to create the audiobook.

Based on the success of the project, it could be extended with image or video processing features in upcoming days.

### Import Libraries and Functions

In [1]:
# basic libraries
import os
import pandas as pd

In [2]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# nlp libraries
import spacy
nlp = spacy.load("en_core_web_lg")

In [4]:
# text-to-speech libraries
from gtts import gTTS
from moviepy.editor import concatenate_audioclips, AudioFileClip
import librosa
import soundfile as sf

### Files and Dataframes Defining

In [5]:
# checking stories
os.listdir("stories")

['beautiful garden.txt', 'cafe at midnight.txt', 'the painted door.txt']

In [6]:
# input title
title = input("Enter text file name:")

Enter text file name: cafe at midnight


In [7]:
# opening text file
story = f"stories/{title}.txt"
with open(story, "r") as f:
    text = f.read()

In [8]:
# create dataframes
df_characters = pd.DataFrame(columns=["cid", "name", "frequency", "features"])
df_lines = pd.DataFrame(columns=["pid", "ndid", "character", "dialogue", "narration"])

# Characters Identification

In [9]:
# finding personal entities from text
entities = nlp(text).ents
personal_entities = []
for entity in entities:
    if entity.label_ == 'PERSON':
        personal_entities.append(entity.text)
personal_entities = list(set(personal_entities))

In [10]:
# adding persons and counts to df_characters
cid_num = 0
for person in personal_entities:
    df_characters = df_characters._append({
        'cid': cid_num, 'name': person, 'frequency': text.count(person), 'features': None}, ignore_index=True)
    cid_num += 1

In [11]:
# characters
df_characters.head(10)

,cid,name,frequency,features
0,0,Alex,7,None


# Lines Identification

In [12]:
# converting text to paragraphs
paragraphs = text.split("\n")
non_empty_paragraphs = list(filter(lambda x: x != '', paragraphs))

In [13]:
# function to identify narrations and dialogues
def identify_narrations_and_dialogues(paragraph):
    """
    :param paragraph: string of paragraph in a story
    :return: list of tuples in (id, name_of_speaker, dialogue, narration) format
    """
    divisions = paragraph.split('"')
    divisions = list(filter(lambda x: x != '', divisions))
    i = 0
    narrations_and_dialogues = []
    for division in divisions:
        start_index = paragraph.find(division)
        end_index = start_index + len(division) - 1
        if '"' in paragraph[:start_index] and '"' in paragraph[end_index:]:
            narrations_and_dialogues.append((i, None, division, None))
            print(division, "D")
        else:
            narrations_and_dialogues.append((i, None, None, division))
        i += 1
    return narrations_and_dialogues


In [14]:
# identifying lines (narrations or dialogues) from each paragraphs
pid_num = 0
for paragraph in non_empty_paragraphs:
    for row in [(pid_num,)+nad for nad in identify_narrations_and_dialogues(paragraph)]:
        print(row)
        df_lines = df_lines._append(pd.Series(row, index=df_lines.columns), ignore_index=True)
    pid_num += 1

(0, 0, None, None, 'Midnight in the heart of the city, the neon signs reflected off the wet pavement as a lone cafÃ© stood open. Inside, Alex sat by the window, sipping coffee and lost in thought.')
(1, 0, None, None, 'The door creaked open, and a mysterious figure entered, their face hidden by the shadows. They took a seat across from Alex without uttering a word.')
Late night for a coffee, isn't it? D
(2, 0, None, None, 'Alex raised an eyebrow. ')
(2, 1, None, "Late night for a coffee, isn't it?", None)
Some stories unfold when the world sleeps. D
(3, 0, None, None, 'The stranger chuckled, a voice tinged with intrigue. ')
(3, 1, None, 'Some stories unfold when the world sleeps.', None)
What kind of stories? D
(4, 0, None, None, 'Intrigued, Alex leaned forward. ')
(4, 1, None, 'What kind of stories?', None)
Stories of the forgotten, the ones that only emerge when the city is draped in silence, D
(5, 0, None, 'Stories of the forgotten, the ones that only emerge when the city is draped 

In [15]:
# lines
df_lines.head(10)

,pid,ndid,character,dialogue,narration
0,0,0,None,None,"Midnight in the heart of the city, the neon si..."
1,1,0,None,None,"The door creaked open, and a mysterious figure..."
2,2,0,None,None,Alex raised an eyebrow.
3,2,1,None,"Late night for a coffee, isn't it?",None
4,3,0,None,None,"The stranger chuckled, a voice tinged with int..."
5,3,1,None,Some stories unfold when the world sleeps.,None
6,4,0,None,None,"Intrigued, Alex leaned forward."
7,4,1,None,What kind of stories?,None
8,5,0,None,"Stories of the forgotten, the ones that only e...",None
9,5,1,None,None,the stranger replied mysteriously.


# Audio Generation

In [16]:
# creating audios
print("Step 1: CONVERSIONS")
for index, row in df_lines.iterrows():
    if row['dialogue'] is not None:
        speech_gtts = gTTS(text=row['dialogue'], lang='en', slow=False, tld='co.in')
    else:
        speech_gtts = gTTS(text=row['narration'], lang='en', slow=False, tld='ie')
    temp_file = f"conversions/{index}.mp3"
    print(f"Line {index+1}/{df_lines.shape[0]} converted.")
    speech_gtts.save(temp_file)
print("Conversions finished.")

Step 1: CONVERSIONS
Line 1/19 converted.
Line 2/19 converted.
Line 3/19 converted.
Line 4/19 converted.
Line 5/19 converted.
Line 6/19 converted.
Line 7/19 converted.
Line 8/19 converted.
Line 9/19 converted.
Line 10/19 converted.
Line 11/19 converted.
Line 12/19 converted.
Line 13/19 converted.
Line 14/19 converted.
Line 15/19 converted.
Line 16/19 converted.
Line 17/19 converted.
Line 18/19 converted.
Line 19/19 converted.
Conversions finished.


In [ ]:
# combining audios
print("Step 2: COMBINATION")
clips = [AudioFileClip(f"conversions/{i}.mp3") for i in range(df_lines.shape[0])]
final_clip = concatenate_audioclips(clips)
print("Combination finished.")

In [ ]:
# adjust speed
print("Step 3: ADJUSTMENT")
final_clip.write_audiofile("conversions/final_slow.mp3")
print("final_slow.mp3 downloaded.")
y, sr = librosa.load("conversions/final_slow.mp3", sr=None)
y_speed = librosa.effects.time_stretch(y, rate=1.25)
print("Speeded over.")

In [ ]:
# removing audios
print("Step 4: REMOVAL")
for i in range(df_lines.shape[0]):
    os.remove(f"conversions/{i}.mp3")
    print(f"Removed {i}.mp3")
os.remove("conversions/final_slow.mp3")
print("Removal over.")

In [ ]:
# downloading final audio
print("Step 5: DOWNLOADING")
final_title = title.replace(" ", "_")
sf.write(f"audiobooks/{final_title}.mp3", y_speed, sr)
print(f"{final_title}.mp3 downloaded")